# Import 

In [ ]:
import os
import skimage.io
import skimage.transform
from scipy import io
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.linear_model import LinearRegression
from captum.attr import IntegratedGradients, LRP, NeuronIntegratedGradients
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tqdm_notebook
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from os.path import join
import torch
from torch import nn
import ast
from ast import literal_eval
import enigmatoolbox.vtk_interface
import statsmodels.api as sm

# Load Integrated gradient values & Calculate Z-score

## IG_Autism

### Load autism_IG

In [ ]:
asd_ig = np.load('asd_IG_467_1.npy')
asd_ig_ = asd_ig.T

### Calculate Z-score

In [ ]:
z = []
z_matrix = []

for i in range(0,47):
    #z.append(asd_ig[i])
    score = zscore(asd_ig[i])
    z_matrix.append(score)
    z=[]
    
z_matrix_asd = np.array(z_matrix)

np.save('asd_zscore_normal(sub).npy',z_matrix_asd)

## IG_Control

### Load Control_IG

In [ ]:
con_ig = np.load('con_IG_9900_420.npy')
con_ig_ = con_ig.T

### Calculate Z-score

In [ ]:
z = []
z_matrix = []

for i in range(0,37):
    #z.append(asd_ig[i])
    score = zscore(con_ig[i])
    z_matrix.append(score)
    z=[]
    
z_matrix_con = np.array(z_matrix)

np.save('con_zscore_normal(sub).npy',z_matrix_con)

### cf) Convert (,9900) to (200,200)

In [ ]:
def convert_dimension(asd_IG):
    
    fdr_200 = np.array(asd_IG)
    fdr_200_matrix = np.zeros((200, 200))
    matrix_200 = np.zeros((200, 200))
    for k in range(0, 100):
        if k == 99 or k == 199:
            fdr_200_matrix[k][k] = 0
        else:
            a = 4950 - ((99 - k) * (100 - k)) / 2
            a = int(a)
            change_matrix = fdr_200[0][a:a + (99 - k)]
            for n in range(0, 99):
                if n == 99 - (k):
                    break
                else:
                    fdr_200_matrix[k][k + n + 1] = change_matrix[n]
            for n in range(0, 99):
                if k == n:
                    fdr_200_matrix[k][k] = 0
    for k in range(100, 200):
        if k == 99 or k == 199:
            fdr_200_matrix[k][k] = 0
        else:
            a = 4950 * 2 - ((99 - (k - 100)) * (100 - (k - 100))) / 2
            a = int(a)
            change_matrix = fdr_200[0][a:a + (99 - (k - 100))]
            for n in range(0, 99):
                if n == 99 - (k - 100):
                    break
                else:
                    fdr_200_matrix[k][k + n + 1] = change_matrix[n]
            for n in range(100, 199):
                if k == n:
                    fdr_200_matrix[k][k] = 0
    matrix_200 = fdr_200_matrix.T + fdr_200_matrix
    return matrix_200

#### convert_autism

In [ ]:
asd_IG_mean = np.mean(z_matrix_asd,axis=0)
asd_IG = np.expand_dims(asd_IG_mean,axis=0)

In [ ]:
matrix_200_asd = convert_dimension(asd_IG)

In [ ]:
np.save('asd_IG_nor_mean(matrix).npy',matrix_200_asd)

#### convert_control

In [ ]:
con_IG_mean = np.mean(z_matrix_con,axis=0)
con_IG = np.expand_dims(con_IG_mean,axis=0)

In [ ]:
matrix_200_con = convert_dimension(con_IG)

In [ ]:
np.save('con_IG_nor_mean(matrix).npy',matrix_200_con)

# Permutation t-test

## Load zscore values

In [ ]:
asd_z = np.load('asd_zscore_normal(sub).npy')
con_z = np.load('con_zscore_normal(sub).npy')

asd_z = asd_z.T
con_z = con_z.T

## Real t-stat value

In [ ]:
# real t stat value
real_t =[]
for it in range(0,9900):
    r_t,r_p=stats.ttest_ind(asd_z[it],con_z[it])
    real_t.append(r_t)

## Permutation t-test

In [ ]:
# permutation t-test
count_list = []
real_t_abs = np.abs(real_t)
for j in range(0,9900):
    total_list = np.concatenate((asd_z[j],con_z[j]))
    total_list_ = total_list.copy()
    G_tvalu = []
    count = 0
    for i in range(0,1000):
        np.random.shuffle(total_list_)
        asd_ran = total_list_[:47]
        con_ran = total_list_[47:]
        f_t, f_p = stats.ttest_ind(asd_ran, con_ran)
        G_tvalu.append(f_t)
    for i in range(0,1000):
        G_tvalu_abs = np.abs(G_tvalu)
        if (real_t_abs[j] < G_tvalu_abs[i]):
            count = count + 1
    count_list.append(count)

In [ ]:
mean = np.mean(G_tvalu)
var = np.var(G_tvalu)

c_list = np.array(count_list)
p_value = c_list/1000  # and np.save('p_value.npy',p_value)

p = p_value.copy()

for i in range(0, 9900):
    if (p[i] < 0.05 and p[i] > 0):
        p[i] = 10

count_10 = 0 # counting the number of value 10
for i in range(0,9900):
    if p[i] == 10:
        count_10 = count_10 + 1

real_t_ = np.array(real_t)

count_nan = 0 # counting the number of nan
for i in range(0, 9900):
    if np.isnan(real_t[i]):
        count_nan = count_nan + 1

In [ ]:
np.save('pvalue_normal.npy',p_value)
np.save('real_t_nor_9900.npy',real_t)

# FDR correction

In [ ]:
# pvalue tuple
pvalue_dic = dict(enumerate(p_value))
pvalue_tuple = list(pvalue_dic.items())


# real_t value tuple
real_t_dic = dict(enumerate(real_t))
real_t_tuple = list(real_t_dic.items())


# real_t nan value
real_nan_valid= []
for i in range(0,9900):
    if np.isnan(real_t_tuple[i][1]):
        real_nan_valid.append(real_t_tuple[i])

# real_t nan keys
real_nan_keys=[]
for j in range(0,np.array(real_nan_valid).shape[0]):
    real_nan_keys.append(real_nan_valid[j][0])

# delete nan keys after copy to p_dic (-> r_dic)
p_dic = pvalue_dic.copy()
for i in real_nan_keys:
    del p_dic[i]

p_dic_value= np.array(list(p_dic.values()))

p_dic_keys =np.array(list(p_dic.keys()))


rejected_n, pvalue_n= statsmodels.stats.multitest.fdrcorrection(p_dic_value)


p_value_dic_nan = dict(zip(p_dic_keys,pvalue_n))
pvalue_tuple_n = list(p_value_dic_nan.items())

fdr_valid_n = []
for i in range(0, np.array(pvalue_tuple_n).shape[0]):
    if (pvalue_tuple_n[i][1] < 0.05):
        fdr_valid_n.append(pvalue_tuple_n[i])

fdr_keys_n = []
for j in range(0, np.array(fdr_valid_n).shape[0]):
    fdr_keys_n.append(fdr_valid_n[j][0])

# save fdr_index(key)
np.save("fdr_index(key).npy",fdr_keys_n)

real_t_valid_n = []
for i in range(0, np.array(fdr_valid_n).shape[0]):
    real_t_valid_n.append(real_t_tuple[fdr_keys_n[i]])

## t value matrix using FDR

In [ ]:
fdr_matrix=np.zeros((1,9900))
for num in range(0,9900):
    for f_n in range(0,np.array(fdr_valid_n).shape[0]):
        if (num==real_t_valid_n[f_n][0]):
            fdr_matrix[0][num]=real_t_valid_n[f_n][1]

In [ ]:
matrix_200 = convert_dimension(fdr_matrix)
np.save('fdr_matrix_normal.npy',fdr_matrix)